# Open AI Playground

## Setup Open AI

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
OPEN_AI_API_KEY = os.getenv('OPEN_AI_API_KEY')
OPEN_AI_ASST_ID = os.getenv('OPEN_AI_ASST_ID')

client = OpenAI(
    api_key=OPEN_AI_API_KEY
)

## Retrieve Assistant

In [ ]:
assistant = client.beta.assistants.retrieve(
    assistant_id=OPEN_AI_ASST_ID
)
assistant

## Create a Thread
Here we can create a thread, think of it as a new "chat". We can add as many messages we want to the thread. We can see the entire thread history of all user and AI repsonse messages.

In [ ]:
thread = client.beta.threads.create()
thread

## Add a Message to the Thread
Here we can add our user defined messages to the thread.

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=(
        f"Create a 2D Maze of with grid dimension of 10."
        f"Once the maze is generated, call the solve maze function to solve the maze."
    )
)
message

## Create Run and Stream Updates
Here we tell the Assistant to run the messages we added to thread. We also check if the run submitted any tool outputs (ie. Function Calls). If a function call is submitted we check which function was called and run our own function to return the result to Open AI.

In [ ]:
import json
from typing_extensions import override
from openai import AssistantEventHandler
from openai.types.beta.threads.run import Run
from maze import solve_maze_bfs, create_maze, highlight_path
from IPython.display import display, Markdown

 
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id
            self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data: Run, run_id):
        tool_outputs = []
            
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "generate_maze":
                generator_inputs: dict = json.loads(tool.function.arguments)
                generator_dim = generator_inputs.get('dimension')
                generated_maze = create_maze(dim=generator_dim)

                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": json.dumps(generated_maze)
                })
            elif tool.function.name == "solve_maze":
                solve_inputs: dict = json.loads(tool.function.arguments)
                solve_maze = solve_inputs.get('grid')
                solved_path = solve_maze_bfs(maze=solve_maze)
                highlighted_maze = highlight_path(solve_maze, solved_path)

                solved_output: dict = {
                    "path_status": "Path Found" if solved_path else "No Path Found",
                    "solution_path": solved_path,
                    "highlighted_maze": highlighted_maze
                }
                tool_outputs.append({
                    "tool_call_id": tool.id,
                    "output": json.dumps(solved_output)
                })
        
        self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            markdown_text = ""
            for text in stream.text_deltas:
                markdown_text += text
                print(text, end="", flush=True)

            display(Markdown(markdown_text))
    
 
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
) as stream:
    stream.until_done()